# Рекомендация тарифов

В нашем распоряжении данные о поведении клиентов (количестве использованных минут, трафике интернета), которые уже перешли на новые тарифы  «Смарт» или «Ультра» (из проекта курса «Статистический анализ данных»).  
**Целью является** - построить модель для решения задачи классификации, которая выберет подходящий тариф. Эта модель будет использована при построении системы, которая способна проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра». Предобработка данных не понадобится, так как ее сделали в предыдущем проекте по «Статистическому анализу данных».  
**Задача**  
Сравнить несколько моделей. Построить модель с максимально большим значением *accuracy*. Доля правильных ответов должна быть не меньше 0.75 на тестовой выборке.

## Откройте и изучите файл

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
df = pd.read_csv('/datasets/users_behavior.csv')

Исследуем данные на наличие пропусков, некорректных данных, типов данных

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [ ]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [ ]:
df.tail()

,calls,minutes,messages,mb_used,is_ultra
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0
3213,80.0,566.09,6.0,29480.52,1


In [ ]:
df.isna().sum()

calls       0
minutes     0
messages    0
mb_used     0
is_ultra    0
dtype: int64

**Вывод**  
Наблюдаем отсутствие пропусков, типы данных - корректные, артефактов не наблюдаем. Данные готовы к использованию.

## Разбейте данные на выборки

Формируем обучающую, валидную и тестовую выборки в соотношении 3:1:1

In [ ]:
df_train, df_valid, df_test = np.split(df.sample(frac=1, random_state=821), [int(.6*len(df)), int(.8*len(df))])
print(df_train.shape)
print(df_test.shape)
print(df_valid.shape)

(1928, 5)
(643, 5)
(643, 5)


Определяем обучающий набор данных и целевой признак, который нужно предсказать по остальным признакам. Под целевым признаком принимаем значения столбца 'is_ultra' - каким тарифом пользовался абонент в течение месяца. В обучающий набор данных включаем столбцы за исключением 'is_ultra'.

In [ ]:
features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']
features_test = df_test.drop(['is_ultra'], axis=1)
target_test = df_test['is_ultra']
features_valid = df_valid.drop(['is_ultra'], axis=1)
target_valid = df_valid['is_ultra']

Проверим наборы данных

In [ ]:
print("Разрешение обучающего датасета на входе:", features_train.shape)
print("Разрешение валидационного датасета на входе:", features_valid.shape)
print("Разрешение тестового датасета на входе:", features_test.shape)
print("Разрешение обучающего датасета на выходе:", target_train.shape)
print("Разрешение валидационного датасета на выходе:", target_valid.shape)
print("Разрешение тестового датасета на выходе:", target_test.shape)

Разрешение обучающего датасета на входе: (1928, 4)
Разрешение валидационного датасета на входе: (643, 4)
Разрешение тестового датасета на входе: (643, 4)
Разрешение обучающего датасета на выходе: (1928,)
Разрешение валидационного датасета на выходе: (643,)
Разрешение тестового датасета на выходе: (643,)


## Исследуйте модели

Ввиду того, что целевой признак является категориальным и имеет два значения, то решается задача о бинарной (двоичной) классификации. Исследуем следующие модели для задач классификации: Дерево решений и Случайный лес. Сравним успешность моделей.

Дерево решений

In [ ]:
best_model1 = None
best_result1 = 0
best_depth1 = 0
best_left = 0
for depth in range(1, 8):
    for leaf in range(2, 70):
        model1 = DecisionTreeClassifier(random_state=12345, max_depth=depth, class_weight = 'balanced', max_leaf_nodes=leaf)
        model1.fit(features_train, target_train) # обучаем модель не тренировочной выборке
        predictions = model1.predict(features_valid) # получаем предсказания модели на валидной выборке
        result1 = accuracy_score(target_valid, predictions) # считаем качество модели на валидной выборке
        if result1 > best_result1:
            best_model1 = model1 # сохраняем наилучшую модель
            best_result1 = result1 #  сохраняем наилучшее значение метрики accuracy на валидационных данных
            best_depth1 = depth
            best_left = leaf
            print("Accuracy модели:", best_result1, "Глубина дерева", best_depth1, "максимальное количество листьев", best_left)
print("Accuracy лучшей модели:", best_result1, "Глубина дерева", best_depth1, "максимальное количество листьев", best_left)

Accuracy модели: 0.7387247278382582 Глубина дерева 1 максимальное количество листьев 2
Accuracy модели: 0.7713841368584758 Глубина дерева 2 максимальное количество листьев 3
Accuracy модели: 0.7853810264385692 Глубина дерева 3 максимальное количество листьев 4
Accuracy модели: 0.7884914463452566 Глубина дерева 5 максимальное количество листьев 11
Accuracy модели: 0.7900466562986003 Глубина дерева 7 максимальное количество листьев 20
Accuracy модели: 0.7916018662519441 Глубина дерева 7 максимальное количество листьев 51
Accuracy лучшей модели: 0.7916018662519441 Глубина дерева 7 максимальное количество листьев 51


**Вывод**  
Проверяла качество модели на валидной выборке изменяя глубину дерева от 1 до 10, расчет показал, что лучшее качество достигается при глубине дерева 7. Доля правильных ответов больше установленной в 0.75 достигается при минимальном количестве деревьев 2.

Случайный лес

In [ ]:
best_model2 = None
best_result2 = 0
best_est2 = 0
best_depths = 0
for est in range(1, 5):
    for depth in range(1, 7):
        model2 = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth = depth) # обучаем модель с заданным количеством деревьев
        model2.fit(features_train,target_train) # обучаем модель на тренировочной выборке
        result2 = model2.score(features_valid, target_valid) # считаем качество модели на валидационной выборке
        if result2 > best_result2:
            best_model2 = model2 # сохраняем наилучшую модель
            best_result2 = result2 #  сохраняем наилучшее значение метрики accuracy на валидационных данных
            best_est2 = est
            best_depths = depth
            print("Accuracy модели на валидационной выборке:", best_result2, "Количество деревьев", best_est2,
                  "Глубина деревьев", best_depths)
print("Accuracy лучшей модели на валидационной выборке:", best_result2, "Количество деревьев", best_est2,
      "Глубина деревьев", best_depths)

Accuracy модели на валидационной выборке: 0.7527216174183515 Количество деревьев 1 Глубина деревьев 1
Accuracy модели на валидационной выборке: 0.7838258164852255 Количество деревьев 1 Глубина деревьев 2
Accuracy модели на валидационной выборке: 0.7884914463452566 Количество деревьев 1 Глубина деревьев 3
Accuracy модели на валидационной выборке: 0.7931570762052877 Количество деревьев 2 Глубина деревьев 5
Accuracy модели на валидационной выборке: 0.7962674961119751 Количество деревьев 2 Глубина деревьев 6
Accuracy модели на валидационной выборке: 0.7993779160186625 Количество деревьев 4 Глубина деревьев 5
Accuracy модели на валидационной выборке: 0.807153965785381 Количество деревьев 4 Глубина деревьев 6
Accuracy лучшей модели на валидационной выборке: 0.807153965785381 Количество деревьев 4 Глубина деревьев 6


**Вывод**  
Проверяла качество модели на валидной выборке изменяя глубину дерева от 1 до 10, расчет показал, что качество модели растет с увеличением деревьев. Доля правильных ответов установленной на уровне 0.75 достигается при минимальном количестве деревьев 2. Лучший результат достигается с количеством деревьев 8.

## Проверьте модель на тестовой выборке

In [ ]:
#проверяем лучшую модель "Дерево решений"
predictions = best_model1.predict(features_test) # получаем предсказания модели на тестовой выборке
result1 = accuracy_score(target_test, predictions) # расчитываем метрику accuracy
print('Accuracy лучшей модели "Дерево решений" на тестовой выборке:', result1)

Accuracy лучшей модели "Дерево решений" на тестовой выборке: 0.7698289269051322


**Вывод**
На тестовой выборке при использовании модели "Дерева решений" получены чуть хуже результаты, чем на валидной выборке.

In [ ]:
#проверяем лучшую модель "Случайный лес"
result2 = best_model2.score(features_test, target_test)  # расчитываем метрику  методом score()
print('Accuracy лучшей модели "Случайный лес" на тестовой выборке:', result2)

Accuracy лучшей модели "Случайный лес" на тестовой выборке: 0.7978227060653188


**Вывод**
На тестовой выборке при использовании модели "Случайный лес" получены получены чуть хуже результаты, чем на валидной выборке.

## Проверка модели на адекватность

Проверяем на адекватность модели путем сравнения качества модели на обучающей и тестовой выборках. Используем лучшую модель, которая сохранена в переменной best_model1 и best_model2.

проверить на адекватность модель можно двуми способами:
    
1. сравнение с гаданием, с "подброшенной монеткой". Вероятность выпадения орла или решки 50%. По сравнению с 50ю проценатми результат в 70 кажется хорошим.
    
2. Какой самый частый класс в наших данных? Ответ: 0, то есть тариф Смарт. Сколько таких тарифов в наших данных? Ответ: 69%. Если мы не будем строить модель, но просто каждому заемщику будем предлагать тариф Смарт, то в скольки процентах случае будем правы? Ответ: в 69%. Вывод: мы не строили модель и у нас точность 69%, а наша модель дает точность в 70%.   

In [ ]:
predictions = best_model1.predict(features_train) # получаем предсказания модели на валидной выборке
accuracy_target = accuracy_score(target_train, predictions)
predictions = best_model1.predict(features_test) # получаем предсказания модели на валидной выборке
accuracy_target_test = accuracy_score(target_test, predictions)
print("Accuracy")
print("Обучающая выборка:", accuracy_target)
print("Тестовая выборка:",accuracy_target_test)

Accuracy
Обучающая выборка: 0.803941908713693
Тестовая выборка: 0.7698289269051322


На тестовой выборке результаты хуже, чем на обущающей. Делаем вывод, что модель переобучена.Результат на тестовой выборке больше установленного 0.75. Удалось достичь accuracy не меньше 0.75

In [ ]:
accuracy_target = best_model2.score(features_train, target_train)
accuracy_target_test = best_model2.score(features_test, target_test)
print("Accuracy")
print("Обучающая выборка:", accuracy_target)
print("Тестовая выборка:",accuracy_target_test)

Accuracy
Обучающая выборка: 0.8189834024896265
Тестовая выборка: 0.7978227060653188


На тестовой выборке результаты хуже, чем на обучающей. Делаем вывод, что модель переобучена. Результат на тестовой выборке больше установленного 0.75. Удалось достичь accuracy не меньше 0.75

**Общий вывод**  
В результате работы были исследованы две модели ("Дерево решений" и "Случайный лес" ) для решения задачи классификации, которые будут выбирать подходящий тариф для пользователей сотового оператора. Согласно данным лучшей оказалась модель "Случайный лес" ее качество, расчитанное на обучающей модели оказалось выше. На тестовой выборке обе модели показали одинаковые результаты. Если адекватность модели определяется минимальным отклонением между значениями accuracy на обущающей и тестовой выбках, то более адекватной оказалась модель "Дерево решений". Заданный уровень качества моделей достигнут и составляет более 0.75. Для использования модели при построении системы, которая способна проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра», рекомендуется модель "Случайный лес" так как при равной метрике качества на тестовой выборке качество модели на обучающей выборке показала выше.